In [2]:
import re, os
from src.graph_parameters import GraphParameters
from src.graph_stats import GraphStats
from src.dag_generational_comparison import DAG_GenerationalComparison
from src.dag import DAG
from src.node import Node
from src.edge import Edge
from src.utils import load_dag

In [3]:
pkl_re = re.compile('.*P[0-9]*\\\\dag.pkl')
data_files = [os.path.join(path, name) for path, _, files in os.walk('.\\data') for name in files]
graph_files = list(filter(pkl_re.match, data_files)) 
print(f"Currently avalible graph files: {graph_files}")

dag_names = []
dags = []

for g in graph_files:
    dag_id = g.split('data\\')[1].split('\\')[0]
    dag_stats = f'results/{dag_id}_dag_with_stats.pkl'
    rec_file = g.split('dag.pkl')[0] + 'reconstruction.npy'
    # gp = GraphParameters(g, rec_file, dag_id=dag_id)
    # gs = GraphStats(gp.dag, dag_id)
    dag_with_stats = load_dag(dag_stats)
    # gs.generate_all_stats()

    dags.append(dag_with_stats)
    dag_names.append(dag_id)

Currently avalible graph files: ['.\\data\\P01\\dag.pkl', '.\\data\\P07\\dag.pkl', '.\\data\\P09\\dag.pkl', '.\\data\\P12\\dag.pkl', '.\\data\\P18\\dag.pkl']


In [30]:
import matplotlib.pyplot as plt
import numpy as np
import csv

class DAG_GenerationalComparison:
    def __init__(self, dags, dag_names, max_gen=8):
        self.dags = dags
        self.dag_names = dag_names
        assert len(dags) == len(dag_names)
        self.max_gen = max_gen

    def __get_plot_comparison(self, data, title):
        figure, axis = plt.subplots(self.max_gen, 1)
        figure.set_size_inches(8, self.max_gen * 4)
        for gen in range(1, self.max_gen+1):
            axis[gen-1].set_title(f'Generation {gen}')
            axis[gen-1].plot(self.dag_names, [data[gen-1][graph] for graph in range(len(self.dags))], 'd')
            
        plt.xlabel('Liver Graph')
        plt.ylabel('Generation')
        plt.savefig(f"results/graph_comparison_{title}")
        plt.clf()

    def __get_boxplot_comparison(self, data, title):
        figure, axis = plt.subplots(self.max_gen, 1)
        figure.set_size_inches(8, self.max_gen * 4)
        plt.setp(axis, xticklabels=self.dag_names)
        for gen in range(1, self.max_gen+1):
            axis[gen-1].set_title(f'Generation {gen}')
            axis[gen-1].boxplot([data[gen-1][graph] for graph in range(len(self.dags))], showfliers=False)
            
        plt.xlabel('Liver Graph')
        plt.ylabel('Generation')
        plt.savefig(f"results/graph_comparison_{title}")
        plt.clf()

    def __generational_comparison(self, data, title, y_label):
        for d in data:
            sd = sum(d)
            for i in range(len(d)):
                d[i] /= sd

        gens = [i+1 for i in range(self.max_gen)]
        plot_data = []
        
        for i,d in enumerate(data):
            plot_data.append(gens)
            plot_data.append(d)

        plt.plot(*plot_data)
        plt.legend(self.dag_names)
        plt.ylabel(y_label)
        plt.xlabel('Generation')
        plt.savefig(f"results/generational_comparison_{title}")
        print(plot_data[1::2])
        with open('GFG', 'w') as f:
            # using csv.writer method from CSV package
            write = csv.writer(f)
            write.writerow([i+1 for i in range(self.max_gen)])
            write.writerows(plot_data[1::2])
        plt.clf()

    def __compare_lengths(self):
        lengths_per_gen_per_graph = [[[] for _ in range(len(self.dags))] for _ in range(self.max_gen)]

        for i, d in enumerate(self.dags):
            lengths_generations = [(edge['length'], edge['generation']) for edge in d.edges]
            for e in lengths_generations:
                if (e[1]-1 < self.max_gen):
                    lengths_per_gen_per_graph[e[1]-1][i].append(e[0])

        edges_per_gen_per_graph = [[len(lengths_per_gen_per_graph[g][i]) for i in range(len(self.dags))] for g in range(self.max_gen)]
        edges_per_graph_per_gen = [[edges_per_gen_per_graph[g][i] for g in range(self.max_gen)] for i in range(len(self.dags))]

        self.__get_boxplot_comparison(lengths_per_gen_per_graph, "lengths_per_generation")
        self.__get_plot_comparison(edges_per_gen_per_graph, "edges_per_generation")
        self.__generational_comparison(edges_per_graph_per_gen, "edges_per_graph_per_gen", "edges / total edges")


    def __compare_diameters(self):
        diameters_per_gen_per_graph = [[[] for _ in range(len(self.dags))] for _ in range(self.max_gen)]
        for i, d in enumerate(self.dags):
            diameters_generations = [(edge['mean_radius'], edge['generation']) for edge in d.edges]
            for e in diameters_generations:
                if (e[1]-1 < self.max_gen):
                    diameters_per_gen_per_graph[e[1]-1][i].append(e[0])

        self.__get_boxplot_comparison(diameters_per_gen_per_graph, "diameters_per_generation")

    def __compare_bifurcation_angles(self):
        bifurcation_angles_per_gen_per_graph = [[[] for _ in range(len(self.dags))] for _ in range(self.max_gen)]
        for i, d in enumerate(self.dags):
            angles_generations = [(edge['relative_angle'] / np.pi * 180, edge['generation']) for edge in d.edges[1:]]
            for e in angles_generations:
                if (e[1]-1 < self.max_gen):
                    bifurcation_angles_per_gen_per_graph[e[1]-1][i].append(e[0])

        self.__get_boxplot_comparison(bifurcation_angles_per_gen_per_graph, "bifurcation_angles_per_generation")

    def __compare_tortuosities(self):
        tortuosities_per_gen_per_graph = [[[] for _ in range(len(self.dags))] for _ in range(self.max_gen)]
        for i, d in enumerate(self.dags):
            tortuosities_generations = [(edge['tortuosity'], edge['generation']) for edge in d.edges]
            for e in tortuosities_generations:
                if (e[1]-1 < self.max_gen):
                    tortuosities_per_gen_per_graph[e[1]-1][i].append(e[0])

        self.__get_boxplot_comparison(tortuosities_per_gen_per_graph, "tortuosities_per_generation")

    def __compare_interstitial_distances(self):
        interstitial_distances_per_gen_per_graph = [[[] for _ in range(len(self.dags))] for _ in range(self.max_gen)]
        for i, d in enumerate(self.dags):
            interstitial_distances_generations = [(edge['interstitial_distance'], edge['generation']) for edge in d.edges]
            for e in interstitial_distances_generations:
                if (e[1]-1 < self.max_gen):
                    interstitial_distances_per_gen_per_graph[e[1]-1][i].append(e[0])

        self.__get_boxplot_comparison(interstitial_distances_per_gen_per_graph, "interstitial_distances_angles_per_generation")

    def compare_all(self):
        self.__compare_lengths()
        # self.__compare_diameters()
        # self.__compare_bifurcation_angles()
        # self.__compare_tortuosities()
        # self.__compare_interstitial_distances()

In [31]:
dag_gen_comparison = DAG_GenerationalComparison(dags, dag_names, 8)
dag_gen_comparison.compare_all()

c:\Users\kopty\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:27: UserWarning: FixedFormatter should only be used together with FixedLocator


[[0.0016025641025641025, 0.008012820512820512, 0.019230769230769232, 0.046474358974358976, 0.09134615384615384, 0.18108974358974358, 0.27884615384615385, 0.3733974358974359], [0.001890359168241966, 0.005671077504725898, 0.015122873345935728, 0.034026465028355386, 0.07750472589792061, 0.15500945179584122, 0.28166351606805295, 0.42911153119092627], [0.004366812227074236, 0.008733624454148471, 0.021834061135371178, 0.048034934497816595, 0.1091703056768559, 0.21397379912663755, 0.2183406113537118, 0.37554585152838427], [0.0026041666666666665, 0.005208333333333333, 0.018229166666666668, 0.033854166666666664, 0.0703125, 0.14583333333333334, 0.2682291666666667, 0.4557291666666667], [0.0013568521031207597, 0.0027137042062415195, 0.013568521031207599, 0.04341926729986431, 0.07327001356852103, 0.1546811397557666, 0.2645861601085482, 0.44640434192673]]


<Figure size 576x2304 with 0 Axes>

<Figure size 576x2304 with 0 Axes>